# Lab 7B: Build Your Own MCP Server

Deploy an MCP server using Azure Functions that calculates ISS position.

This uses the `mcpToolTrigger` binding - no HTTP-level MCP implementation needed!

## Part 1: Configuration

In [ ]:
import os, json, subprocess, math, hashlib, requests
from datetime import datetime, timezone
from dotenv import load_dotenv

load_dotenv("../../.env")

RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP", "foundry-mcp-rg")
LOCATION = os.environ.get("LOCATION", "eastus")

# Use deterministic hash (subscription + RG) for cross-user uniqueness
def get_unique_suffix():
    result = subprocess.run(["az", "account", "show", "--query", "id", "-o", "tsv"], capture_output=True, text=True)
    sub_id = result.stdout.strip() if result.returncode == 0 else "default"
    return hashlib.md5(f"{sub_id}-{RESOURCE_GROUP}".encode()).hexdigest()[:6]

UNIQUE_SUFFIX = os.environ.get("UNIQUE_SUFFIX", get_unique_suffix())
FUNC_APP_NAME = f"iss-mcp-{UNIQUE_SUFFIX}"
STORAGE_NAME = f"issmcp{UNIQUE_SUFFIX}".replace("-", "")[:24]

SPOKE_ENDPOINT = os.environ.get("SPOKE_ENDPOINT", "")
SPOKE_PROJECT = os.environ.get("SPOKE_PROJECT", "")
APIM_CONNECTION = os.environ.get("APIM_CONNECTION", "")
MODEL_NAME = os.environ.get("MODEL_NAME", "gpt-4o")

if SPOKE_ENDPOINT:
    account_host = SPOKE_ENDPOINT.replace("https://", "").replace(".cognitiveservices.azure.com/", "")
    PROJECT_ENDPOINT = f"https://{account_host}.services.ai.azure.com/api/projects/{SPOKE_PROJECT}"
    GATEWAY_MODEL = f"{APIM_CONNECTION}/{MODEL_NAME}"
else:
    PROJECT_ENDPOINT = ""
    GATEWAY_MODEL = ""

result = subprocess.run(["az", "account", "show", "-o", "json"], capture_output=True, text=True)
SUBSCRIPTION_ID = json.loads(result.stdout)["id"] if result.returncode == 0 else None

print(f"Function App: {FUNC_APP_NAME}")
print(f"Storage:      {STORAGE_NAME}")

## Part 2: Create MCP Server with Built-in Triggers

Using `mcpToolTrigger` - the tools are defined via decorators, not HTTP handlers!

In [2]:
import os
os.makedirs("iss-mcp", exist_ok=True)
print("Created: iss-mcp/")

Created: iss-mcp/


In [3]:
%%writefile iss-mcp/requirements.txt
azure-functions

Writing iss-mcp/requirements.txt


In [4]:
%%writefile iss-mcp/host.json
{
  "version": "2.0",
  "extensionBundle": {
    "id": "Microsoft.Azure.Functions.ExtensionBundle.Experimental",
    "version": "[4.*, 5.0.0)"
  }
}

Writing iss-mcp/host.json


In [5]:
%%writefile iss-mcp/function_app.py
import json
import math
from datetime import datetime, timezone
import azure.functions as func

app = func.FunctionApp(http_auth_level=func.AuthLevel.FUNCTION)

# ISS Orbital Constants
EARTH_RADIUS_KM = 6371.0
ISS_ALTITUDE_KM = 408.0
ISS_ORBITAL_PERIOD_MIN = 92.65
ISS_INCLINATION_DEG = 51.6
ISS_VELOCITY_KMS = 7.66
REFERENCE_EPOCH = datetime(2026, 1, 1, 0, 0, 0, tzinfo=timezone.utc)
REFERENCE_LONGITUDE = -80.0

def get_cardinal(h):
    return ["N","NE","E","SE","S","SW","W","NW"][round(h/45)%8]

def calc_position():
    now = datetime.now(timezone.utc)
    mins = (now - REFERENCE_EPOCH).total_seconds() / 60.0
    phase = (mins / ISS_ORBITAL_PERIOD_MIN % 1.0) * 2 * math.pi
    inc = math.radians(ISS_INCLINATION_DEG)
    lat = math.degrees(math.asin(math.sin(inc) * math.sin(phase)))
    drift = (360/ISS_ORBITAL_PERIOD_MIN - 360/1436) * mins
    lon_off = math.degrees(math.atan2(math.cos(inc)*math.sin(phase), math.cos(phase)))
    lon = (REFERENCE_LONGITUDE - drift + lon_off) % 360
    if lon > 180: lon -= 360
    return lat, lon, now, phase

# Tool 1: Get ISS Position
@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="get_iss_position",
    description="Get the current position of the International Space Station",
    toolProperties="[]"
)
def get_iss_position(context) -> str:
    lat, lon, now, _ = calc_position()
    return json.dumps({
        "latitude": round(lat, 4),
        "longitude": round(lon, 4),
        "altitude_km": ISS_ALTITUDE_KM,
        "timestamp": now.isoformat()
    })

# Tool 2: Get ISS Velocity
@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="get_iss_velocity",
    description="Get the current velocity and heading of the ISS",
    toolProperties="[]"
)
def get_iss_velocity(context) -> str:
    lat, _, _, phase = calc_position()
    lat_r, inc_r = math.radians(lat), math.radians(ISS_INCLINATION_DEG)
    if abs(lat) < ISS_INCLINATION_DEG:
        heading = math.degrees(math.acos(max(-1, min(1, math.cos(lat_r)/math.cos(inc_r)))))
        if math.pi/2 < phase < 3*math.pi/2: heading = 180 - heading
    else:
        heading = 90 if lat > 0 else 270
    return json.dumps({
        "velocity_kms": ISS_VELOCITY_KMS,
        "velocity_mph": round(ISS_VELOCITY_KMS * 2236.94),
        "heading_degrees": round(heading, 1),
        "direction": get_cardinal(heading)
    })

# Tool 3: Get Orbital Info
@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="get_orbital_info",
    description="Get ISS orbital parameters",
    toolProperties="[]"
)
def get_orbital_info(context) -> str:
    return json.dumps({
        "altitude_km": ISS_ALTITUDE_KM,
        "orbital_period_minutes": ISS_ORBITAL_PERIOD_MIN,
        "inclination_degrees": ISS_INCLINATION_DEG,
        "velocity_kms": ISS_VELOCITY_KMS,
        "orbits_per_day": round(1440 / ISS_ORBITAL_PERIOD_MIN, 2)
    })

# Tool 4: Check Visibility
visibility_props = json.dumps([
    {"propertyName": "latitude", "propertyType": "number", "description": "Observer latitude"},
    {"propertyName": "longitude", "propertyType": "number", "description": "Observer longitude"}
])

@app.generic_trigger(
    arg_name="context",
    type="mcpToolTrigger",
    toolName="get_iss_visibility",
    description="Check if the ISS is visible from a given location",
    toolProperties=visibility_props
)
def get_iss_visibility(context) -> str:
    args = json.loads(context).get("arguments", {})
    obs_lat = args.get("latitude", 0)
    obs_lon = args.get("longitude", 0)
    
    iss_lat, iss_lon, _, _ = calc_position()
    lat1, lat2 = math.radians(obs_lat), math.radians(iss_lat)
    dlon = math.radians(iss_lon - obs_lon)
    cos_d = max(-1, min(1, math.sin(lat1)*math.sin(lat2) + math.cos(lat1)*math.cos(lat2)*math.cos(dlon)))
    dist = math.acos(cos_d) * EARTH_RADIUS_KM
    bearing = (math.degrees(math.atan2(
        math.sin(dlon)*math.cos(lat2),
        math.cos(lat1)*math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(dlon)
    )) + 360) % 360
    
    return json.dumps({
        "is_visible": dist < 2500,
        "distance_km": round(dist, 1),
        "direction": get_cardinal(bearing),
        "iss_latitude": round(iss_lat, 4),
        "iss_longitude": round(iss_lon, 4)
    })

Writing iss-mcp/function_app.py


## Part 3: Deploy to Azure

In [6]:
!az group create --name {RESOURCE_GROUP} --location {LOCATION} -o none && echo "Resource group ready"

Resource group ready


In [ ]:
%%bash -s "$RESOURCE_GROUP" "$STORAGE_NAME" "$LOCATION"
az storage account show -g "$1" -n "$2" -o none 2>/dev/null || \
az storage account create -g "$1" -n "$2" -l "$3" --sku Standard_LRS -o none && echo "Storage ready: $2"

In [ ]:
%%bash -s "$RESOURCE_GROUP" "$FUNC_APP_NAME" "$STORAGE_NAME" "$LOCATION" "$SUBSCRIPTION_ID"
RG=$1; APP=$2; STOR=$3; LOC=$4; SUB=$5

if ! az functionapp show -g "$RG" -n "$APP" -o none 2>/dev/null; then
    az storage container create --account-name "$STOR" --name deployments --auth-mode login -o none 2>/dev/null || true
    az functionapp create -g "$RG" -n "$APP" --storage-account "$STOR" --runtime python --runtime-version 3.11 \
        --flexconsumption-location "$LOC" --deployment-storage-container-name deployments \
        --deployment-storage-auth-type SystemAssignedIdentity -o none
    
    PID=$(az functionapp identity show -g "$RG" -n "$APP" --query principalId -o tsv)
    SCOPE="/subscriptions/$SUB/resourceGroups/$RG/providers/Microsoft.Storage/storageAccounts/$STOR"
    for ROLE in "Storage Blob Data Contributor" "Storage Queue Data Contributor" "Storage Table Data Contributor"; do
        az role assignment create --assignee "$PID" --role "$ROLE" --scope "$SCOPE" -o none 2>/dev/null || true
    done
    
    az functionapp config appsettings set -g "$RG" -n "$APP" --settings \
        "AzureWebJobsStorage__accountName=$STOR" \
        "AzureWebJobsStorage__blobServiceUri=https://${STOR}.blob.core.windows.net/" \
        "AzureWebJobsStorage__queueServiceUri=https://${STOR}.queue.core.windows.net/" \
        "AzureWebJobsStorage__tableServiceUri=https://${STOR}.table.core.windows.net/" -o none
    az functionapp config appsettings delete -g "$RG" -n "$APP" --setting-names AzureWebJobsStorage -o none 2>/dev/null || true
    sleep 30
fi
echo "Function App ready: $APP"

In [58]:
%%bash -s "$RESOURCE_GROUP" "$FUNC_APP_NAME"
cd iss-mcp && zip -rq ../iss-mcp.zip . && cd ..
az functionapp deployment source config-zip -g "$1" -n "$2" --src iss-mcp.zip -o none
echo "Deployed"

Deployed


In [ ]:
import time; time.sleep(30)

FUNC_HOST = f"{FUNC_APP_NAME}.azurewebsites.net"
result = subprocess.run(["az", "functionapp", "keys", "list", "-g", RESOURCE_GROUP, "-n", FUNC_APP_NAME, 
                         "--query", "masterKey", "-o", "tsv"], capture_output=True, text=True)
FUNC_KEY = result.stdout.strip()

# Built-in MCP endpoint path
MCP_URL = f"https://{FUNC_HOST}/runtime/webhooks/mcp"

print(f"MCP URL: {MCP_URL}")
print(f"Key: {FUNC_KEY[:20]}..." if FUNC_KEY else "Key not available yet")

## Part 4: Test the MCP Server

The built-in MCP endpoint is at `/runtime/webhooks/mcp/sse` for SSE connections.

In [ ]:
# Test the SSE endpoint info
sse_url = f"{MCP_URL}/sse?code={FUNC_KEY}"
print(f"SSE Endpoint: {sse_url[:80]}...")
print(f"\nUse this URL with MCP Inspector or Foundry agents")

## Part 5: Connect to Foundry Agent

In [39]:
!pip install azure-ai-projects azure-ai-agents azure-identity -q

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool
from azure.identity import DefaultAzureCredential
from openai.types.responses.response_input_param import McpApprovalResponse

MCP_SSE_URL = f"{MCP_URL}/sse?code={FUNC_KEY}"
print(f"MCP SSE: {MCP_SSE_URL[:60]}...")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
MCP SSE: https://iss-mcp-489e2e.azurewebsites.net/runtime/webhooks/mc...


In [40]:
project_client = AIProjectClient(credential=DefaultAzureCredential(), endpoint=PROJECT_ENDPOINT)
openai_client = project_client.get_openai_client()

agent = project_client.agents.create_version(
    agent_name="iss-tracker",
    definition=PromptAgentDefinition(
        model=GATEWAY_MODEL,
        instructions="You track the ISS. Use the MCP tools to answer questions about ISS position, velocity, visibility, and orbital parameters.",
        tools=[MCPTool(server_label="iss_mcp", server_url=MCP_SSE_URL, require_approval="always")],
    )
)
print(f"Agent: {agent.name}")

Agent: iss-tracker


In [59]:
def ask(question):
    conv = openai_client.conversations.create()
    resp = openai_client.responses.create(
        conversation=conv.id, input=question,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}}
    )
    approvals = [McpApprovalResponse(type="mcp_approval_response", approval_request_id=i.id, approve=True) 
                 for i in resp.output if i.type == "mcp_approval_request" and i.id]
    if approvals:
        resp = openai_client.responses.create(
            input=approvals, previous_response_id=resp.id,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}}
        )
    return resp.output_text

print(ask("Where is the ISS right now?"))

The International Space Station (ISS) is currently located at approximately latitude 10.25° and longitude -164.82°, at an altitude of about 408 km above the Earth.


In [60]:
print(ask("Can I see the ISS from Seattle (47.6, -122.3)?"))

Currently, the ISS is not visible from Seattle (47.6, -122.3). It is approximately 5599 kilometers away towards the southwest direction at the moment.


In [61]:
print(ask("What are the ISS orbital parameters?"))

The International Space Station (ISS) has the following orbital parameters:
- Altitude: Approximately 408 km above the Earth
- Orbital Period: About 92.65 minutes per orbit
- Inclination: 51.6 degrees
- Velocity: Around 7.66 km/s
- Orbits per day: Approximately 15.54 orbits


## Cleanup

In [ ]:
# Uncomment to delete resources
# !az functionapp delete -g {RESOURCE_GROUP} -n {FUNC_APP_NAME} -y
# !az storage account delete -g {RESOURCE_GROUP} -n {STORAGE_NAME} -y